# Limpieza de los datos

In [1]:
import pandas as pd

In [2]:
# aquí están los datos en local
data_path = '/Users/cbautistap/Dropbox/MCD/Cursos/primavera-2021/dpa/data/Food_Inspections.csv'

In [3]:
# cargamos datos, convertimos a formato datetime la columna 'Inspection Date' y queda con formato yy/mm/dd
data = pd.read_csv(data_path, parse_dates=['Inspection Date'])

In [4]:
data.shape

(215130, 17)

In [5]:
#data['month'] = data['Inspection Date'].dt.month

In [6]:
# creamos función para estandarizar el nombre de las columnas.
def standarize_column_names(data, excluded_punctuation=".,-*¿?¡!#"):
    data.columns = data.columns.str.lower().str.replace(" ", "_")
    for ch in excluded_punctuation:
        data.columns = data.columns.str.replace(ch, "")

In [7]:
standarize_column_names(data)

/Users/cbautistap/.pyenv/versions/itam_dpa/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """


In [8]:
# cambiamos el nombre de la columna license
data=data.rename(columns={'license_':'license'})
data.head(2)

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2472391,PHILZ COFFEE,PHILZ COFFEE,2766979.0,NaN,Risk 2 (Medium),1555 N WELLS ST,CHICAGO,IL,60610.0,2021-01-07,License,Not Ready,NaN,41.910736,-87.634551,"(-87.63455069145124, 41.910736031820235)"
1,2453551,BIAN,BIAN,2744753.0,Restaurant,Risk 1 (High),600 W CHICAGO AVE,CHICAGO,IL,60654.0,2020-10-20,License,Pass,NaN,41.896585,-87.642996,"(-87.64299618172501, 41.896585191199556)"


In [9]:
data.dtypes

inspection_id               int64
dba_name                   object
aka_name                   object
license                   float64
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                       float64
inspection_date    datetime64[ns]
inspection_type            object
results                    object
violations                 object
latitude                  float64
longitude                 float64
location                   object
dtype: object

Queremos convertir license y zip a string. Si lo convertimos directo astype(str) queda con decimales, por lo tanto convertimos primero a formato int. Sin embargo, ambas columnas tienen NAs por lo que una solución es llenarlos con 0 y ahora sí convertir a string. QUIZÁ NO HAY PROBLEMA TENERLOS EN STRING CON EL DECIMAL XXXX.0

In [10]:
data.isna().sum()

inspection_id          0
dba_name               0
aka_name            2487
license               17
facility_type       4892
risk                  69
address                0
city                 158
state                 50
zip                   52
inspection_date        0
inspection_type        1
results                0
violations         57447
latitude             710
longitude            710
location             710
dtype: int64

In [11]:
#Subset de data frame con missing values en cualquier columna
null_data = data[data.isnull().any(axis=1)]
null_data.head()

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2472391,PHILZ COFFEE,PHILZ COFFEE,2766979.0,NaN,Risk 2 (Medium),1555 N WELLS ST,CHICAGO,IL,60610.0,2021-01-07,License,Not Ready,NaN,41.910736,-87.634551,"(-87.63455069145124, 41.910736031820235)"
1,2453551,BIAN,BIAN,2744753.0,Restaurant,Risk 1 (High),600 W CHICAGO AVE,CHICAGO,IL,60654.0,2020-10-20,License,Pass,NaN,41.896585,-87.642996,"(-87.64299618172501, 41.896585191199556)"
2,2386633,JIN JU,JIN JU,27137.0,Restaurant,Risk 1 (High),5203 N CLARK ST,CHICAGO,IL,60640.0,2020-08-28,Canvass,No Entry,NaN,41.976301,-87.668276,"(-87.66827593789948, 41.97630115368914)"
3,2386595,LA BIZNAGA #2,LA BIZNAGA #2,2708992.0,NaN,Risk 1 (High),2949 W BELMONT AVE,CHICAGO,IL,60618.0,2020-08-27,Complaint,No Entry,NaN,41.939256,-87.702270,"(-87.70226967930802, 41.939255926667535)"
4,2386523,MC DONALD'S,MC DONALD'S,2560785.0,Restaurant,Risk 2 (Medium),6336 S ASHLAND AVE,CHICAGO,IL,60636.0,2020-08-26,Short Form Complaint,Pass,NaN,41.778361,-87.664337,"(-87.66433686342792, 41.77836097603761)"


### Queremos entender qué tipo de observaciones son las que tienen missing values, con el objetivo de encontrar algún patrón o algo que nos permita imputar de mejor manera. 

Para eso, haremos subset de missing values por columna. Empezamos con las que tienen mayor NaN.

Nota. La columna de aka_name no la exploraremos. Es lógico que haya mv para esta columna, además, no la utilizaremos para el modelo

In [12]:
#queremos ver que observaciones son las que tienen missing values en la columna 'violations'
null_violations = null_data[null_data.violations.isnull()]
null_violations.shape

(57447, 17)

In [13]:
#Missing values en la columna 'violations'
null_data[null_data.violations.isnull()].groupby('results').count()

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,violations,latitude,longitude,location
results,,,,,,,,,,,,,,,,
Business Not Located,72,72,57,72,45,64,72,70,72,70,72,72,0,69,69,69
Fail,3183,3183,2989,3182,2842,3162,3183,3170,3183,3178,3183,3183,0,3168,3168,3168
No Entry,6796,6796,6709,6796,6725,6789,6796,6790,6793,6795,6796,6796,0,6794,6794,6794
Not Ready,2226,2226,2172,2226,2123,2210,2226,2212,2222,2223,2226,2226,0,2218,2218,2218
Out of Business,18677,18677,18098,18676,14638,18673,18677,18658,18674,18663,18677,18677,0,18641,18641,18641
Pass,25925,25925,25491,25922,25729,25914,25925,25895,25913,25916,25925,25924,0,25799,25799,25799
Pass w/ Conditions,568,568,563,568,562,568,568,568,567,568,568,568,0,567,567,567


Agrupando por Results, el 89.4% de los NaN de la columna 'Violations' están en 'Pass', 'Out of Business' y 'No Entry'. Tiene sentido que los que pasaron la inspección no tengan violaciones, asimismo, que los que ya estén cerrados y en los cuales no se entró, no tengan violaciones.

Por otro lado, solo el 5% de los mv de la columna 'Violations' son de resultados 'Fail' (que representan 1.47% del total de observaciones).
Dentro de éstas, el 62% son de Tipo de Inspección 'License'

In [14]:
results_fail = null_violations['results']=='Fail'

In [15]:
null_violations[results_fail].groupby('inspection_type').count()

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,results,violations,latitude,longitude,location
inspection_type,,,,,,,,,,,,,,,,
Business Not Located,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
Canvass,462,462,453,462,340,462,462,462,462,462,462,462,0,461,461,461
Canvass Re-Inspection,16,16,16,16,16,16,16,16,16,16,16,16,0,15,15,15
Complaint,128,128,118,128,117,125,128,127,128,128,128,128,0,128,128,128
Complaint Re-Inspection,6,6,6,6,6,6,6,6,6,6,6,6,0,6,6,6
Complaint-Fire,22,22,22,22,20,22,22,22,22,22,22,22,0,22,22,22
Consultation,45,45,42,45,41,43,45,45,45,45,45,45,0,45,45,45
Duplicated,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
Illegal Operation,1,1,0,1,1,0,1,1,1,1,1,1,0,1,1,1


In [16]:
# MV de columna 'Facility Type'
null_data[null_data.facility_type.isnull()].groupby('results').count()

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,violations,latitude,longitude,location
results,,,,,,,,,,,,,,,,
Business Not Located,27,27,15,27,0,19,27,25,27,25,27,27,0,24,24,24
Fail,368,368,310,368,0,354,368,367,368,367,368,368,27,367,367,367
No Entry,71,71,54,71,0,64,71,71,71,71,71,71,0,71,71,71
Not Ready,103,103,72,103,0,87,103,100,103,103,103,103,0,103,103,103
Out of Business,4040,4040,3996,4040,0,4036,4040,4039,4040,4040,4040,4040,1,4034,4034,4034
Pass,245,245,235,245,0,240,245,243,245,245,245,245,49,243,243,243
Pass w/ Conditions,38,38,37,38,0,38,38,38,38,38,38,38,32,36,36,36


El 84% de los NaN en la columna de Facility Type corresponden a results 'Out of business', 'Business Not Located' y 'No Entry'

In [17]:
# MV de columna 'location'
null_data[null_data.location.isnull()].groupby('results').count()

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,violations,latitude,longitude,location
results,,,,,,,,,,,,,,,,
Business Not Located,3,3,1,3,0,1,3,3,3,3,3,3,0,0,0,0
Fail,145,145,143,145,144,145,145,145,145,145,145,145,130,0,0,0
No Entry,3,3,3,3,3,3,3,3,3,3,3,3,1,0,0,0
Not Ready,8,8,8,8,8,8,8,7,8,7,8,8,0,0,0,0
Out of Business,36,36,35,36,30,36,36,35,36,35,36,36,0,0,0,0
Pass,432,432,426,432,430,432,432,431,432,431,432,432,306,0,0,0
Pass w/ Conditions,83,83,82,83,81,83,83,83,83,83,83,83,82,0,0,0


En total son 710 observaciones con mv en ubicación (que coinciden con coordenadas geográficas). La mayoría son de establecimientos que pasaron la visita. Asimismo, destaca que por facility type, la mayoría son de tipo school y también incluye algunos establecimientos móviles.

In [18]:
# MV de columna 'location', agrupando con columna 'facility_type'
null_data[null_data.location.isnull()].groupby('facility_type').count()

,inspection_id,dba_name,aka_name,license,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
facility_type,,,,,,,,,,,,,,,,
BREWERY,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0
CHURCH/SPECIAL EVENT,3,3,0,3,3,3,3,3,3,3,3,3,3,0,0,0
Daycare Combo 1586,1,1,0,1,1,1,1,1,1,1,1,1,0,0,0,0
Food Vending Machines,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0
GAS STATION,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0
Golden Diner,9,9,9,9,9,9,8,9,8,9,9,9,3,0,0,0
Grocery Store,25,25,25,25,25,25,25,25,25,25,25,25,16,0,0,0
HIGH SCHOOL KITCHEN,3,3,3,3,3,3,3,3,3,3,3,3,2,0,0,0
Hospital,10,10,10,10,10,10,10,10,10,10,10,10,6,0,0,0


In [19]:
# MV de columna 'risk', agrupando por 'results'
null_data[null_data.risk.isnull()].groupby('results').count()

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,violations,latitude,longitude,location
results,,,,,,,,,,,,,,,,
Business Not Located,8,8,2,8,0,0,8,8,8,8,8,8,0,6,6,6
Fail,22,22,15,22,8,0,22,22,22,22,22,22,1,22,22,22
No Entry,7,7,2,7,0,0,7,7,7,7,7,7,0,7,7,7
Not Ready,16,16,9,16,0,0,16,14,16,16,16,16,0,16,16,16
Out of Business,4,4,3,4,0,0,4,4,4,4,4,4,0,4,4,4
Pass,12,12,8,12,7,0,12,11,12,12,12,12,1,12,12,12


In [20]:
# MV de columna 'risk', agrupando por 'inspection_type'
null_data[null_data.risk.isnull()].groupby('inspection_type').count()

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,results,violations,latitude,longitude,location
inspection_type,,,,,,,,,,,,,,,,
Canvass,3,3,2,3,0,0,3,3,3,3,3,3,0,3,3,3
Complaint,10,10,4,10,3,0,10,10,10,10,10,10,1,9,9,9
Complaint-Fire,1,1,0,1,0,0,1,1,1,1,1,1,0,1,1,1
Consultation,5,5,4,5,4,0,5,5,5,5,5,5,1,5,5,5
Illegal Operation,1,1,0,1,1,0,1,1,1,1,1,1,0,1,1,1
License,36,36,20,36,2,0,36,34,36,36,36,36,0,35,35,35
Non-Inspection,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1
Out of Business,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1
Pre-License Consultation,3,3,2,3,2,0,3,3,3,3,3,3,0,3,3,3


## Por lo pronto, para proseguir con limpieza de datos, se decide reemplazar los MV con 0 para variables numéricas y con el valor 'na' para variables de texto.

### En primer lugar, convertimos a variables string las de license y zip 

In [21]:
# Reemplazamos con 0 los mv en estas dos columnas
data['license']=data['license'].fillna(0)
data['zip']=data['zip'].fillna(0)

In [22]:
data.isna().sum()

inspection_id          0
dba_name               0
aka_name            2487
license                0
facility_type       4892
risk                  69
address                0
city                 158
state                 50
zip                    0
inspection_date        0
inspection_type        1
results                0
violations         57447
latitude             710
longitude            710
location             710
dtype: int64

In [23]:
# convertimos columnas 'license' y 'zip'. Primero a int y luego a str
data = data.astype({"license": 'int', "zip": 'int'})
data = data.astype({"license": 'str', "zip": 'str'})
data.dtypes

inspection_id               int64
dba_name                   object
aka_name                   object
license                    object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
violations                 object
latitude                  float64
longitude                 float64
location                   object
dtype: object

In [24]:
data.head(2)

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2472391,PHILZ COFFEE,PHILZ COFFEE,2766979,NaN,Risk 2 (Medium),1555 N WELLS ST,CHICAGO,IL,60610,2021-01-07,License,Not Ready,NaN,41.910736,-87.634551,"(-87.63455069145124, 41.910736031820235)"
1,2453551,BIAN,BIAN,2744753,Restaurant,Risk 1 (High),600 W CHICAGO AVE,CHICAGO,IL,60654,2020-10-20,License,Pass,NaN,41.896585,-87.642996,"(-87.64299618172501, 41.896585191199556)"


In [25]:
# Hacemos subset de las columnas que utilizaremos para el modelo
data = data[['inspection_id', 'facility_type', 'risk', 'zip', 'inspection_date','inspection_type', 'results', 'violations', 'latitude', 'longitude']]
data.head()

,inspection_id,facility_type,risk,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2472391,NaN,Risk 2 (Medium),60610,2021-01-07,License,Not Ready,NaN,41.910736,-87.634551
1,2453551,Restaurant,Risk 1 (High),60654,2020-10-20,License,Pass,NaN,41.896585,-87.642996
2,2386633,Restaurant,Risk 1 (High),60640,2020-08-28,Canvass,No Entry,NaN,41.976301,-87.668276
3,2386595,NaN,Risk 1 (High),60618,2020-08-27,Complaint,No Entry,NaN,41.939256,-87.702270
4,2386523,Restaurant,Risk 2 (Medium),60636,2020-08-26,Short Form Complaint,Pass,NaN,41.778361,-87.664337


In [26]:
data.isnull().sum()

inspection_id          0
facility_type       4892
risk                  69
zip                    0
inspection_date        0
inspection_type        1
results                0
violations         57447
latitude             710
longitude            710
dtype: int64

In [27]:
data[['facility_type', 'risk', 'inspection_type','violations']]=data[['facility_type', 'risk', 'inspection_type','violations']].fillna('na')

In [28]:
data.isnull().sum()

inspection_id        0
facility_type        0
risk                 0
zip                  0
inspection_date      0
inspection_type      0
results              0
violations           0
latitude           710
longitude          710
dtype: int64

In [29]:
# Cambiamos valor de results a 'Pass' y 'Not Pass' para cualquier otro
data['results'].mask(data['results'] != 'Pass', other = 'Not Pass', inplace=True)

In [30]:
data.groupby('results').count()

,inspection_id,facility_type,risk,zip,inspection_date,inspection_type,violations,latitude,longitude
results,,,,,,,,,
Not Pass,102489,102489,102489,102489,102489,102489,102489,102211,102211
Pass,112641,112641,112641,112641,112641,112641,112641,112209,112209


Ahora, estandarizamos los valores de las columnas (quitamos acentos y eliminamos caracteres especiales)

In [31]:
def standarize_column_strings(data, columns, excluded_punctuation=".,-*'¿?¡!()", gap_punct="/\|"):
    for col in columns:
        data[col] = data[col].apply(lambda x: x.lower().replace(" ", "_"))
        data[col] = data[col].apply(lambda x: x.lower().replace("á", "a"))
        data[col] = data[col].apply(lambda x: x.lower().replace("é", "e"))
        data[col] = data[col].apply(lambda x: x.lower().replace("í", "i"))
        data[col] = data[col].apply(lambda x: x.lower().replace("ó", "o"))
        data[col] = data[col].apply(lambda x: x.lower().replace("ú", "u"))
        data[col] = data[col].apply(lambda x: x.lower().replace("ü", "u"))
        for ch in excluded_punctuation:
            data[col] = data[col].apply(lambda x: x.replace(ch, ""))
        for ch in gap_punct:
            data[col] = data[col].apply(lambda x: x.replace(ch,"_"))

In [32]:
#def standarize_column_strings(data, columns, excluded_punctuation=".,-*¿?¡!"):
#    for col in columns:
#        data[col] = data[col].apply(lambda x: x.lower().replace(" ", "_"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("á", "a"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("é", "e"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("í", "i"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("ó", "o"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("ú", "u"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("ü", "u"))
#        for ch in excluded_punctuation:
#            data[col] = data[col].apply(lambda x: x.replace(ch, ""))

In [33]:
#Convertimos variables de texto a formato estándar
col_text = data.select_dtypes(include=['object']).columns.to_list()
standarize_column_strings(data, col_text)
data.head()

,inspection_id,facility_type,risk,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2472391,na,risk_2_medium,60610,2021-01-07,license,not_pass,na,41.910736,-87.634551
1,2453551,restaurant,risk_1_high,60654,2020-10-20,license,pass,na,41.896585,-87.642996
2,2386633,restaurant,risk_1_high,60640,2020-08-28,canvass,not_pass,na,41.976301,-87.668276
3,2386595,na,risk_1_high,60618,2020-08-27,complaint,not_pass,na,41.939256,-87.702270
4,2386523,restaurant,risk_2_medium,60636,2020-08-26,short_form_complaint,pass,na,41.778361,-87.664337


In [34]:
# Función para reemplazar el valor de una columna específica
def replace_column_value(data, column, search_string, replace_string):
    data[column] = data[column].apply(lambda x: x.replace(search_string, replace_string))

In [55]:
replace_column_value(data,'facility_type', 'restuarant', 'restaurant')
replace_column_value(data,'facility_type', 'theatre', 'theater')
replace_column_value(data,'facility_type', 'herabal', 'herbal')
replace_column_value(data,'facility_type', 'day_care', 'daycare')
replace_column_value(data,'facility_type', 'long_term', 'longterm')


#'theatre', 'theater')) %>% 
#  mutate_at(vars(facility_type), ~str_replace_all(., 'herabal', 'herbal')) %>% 
#  mutate_at(vars(facility_type), ~str_replace_all(., 'day care', 'daycare')) %>% 
#  mutate_at(vars(facility_type), ~str_replace_all(., 'long term', 'longterm'))

In [86]:
data.loc[data['facility_type'].str.contains('childern|children|1023|5_years_old', case=False, na=None), 'facility_type'] = 'childrens_service_facility'
data.loc[data['facility_type'].str.contains('conv|mart|gas_station_store', case=False, na=None), 'facility_type'] = 'convenience_store'
data.loc[data['facility_type'].str.contains('assis|longterm|nursing|supportive', case=False, na=None), 'facility_type'] = 'assisted_living'
data.loc[data['facility_type'].str.contains('herbal_life|herbalife|herbalcal', case=False, na=None), 'facility_type'] = 'herbalife'
data.loc[data['facility_type'].str.contains('after_school', case=False, na=None), 'facility_type'] = 'after_school'
data.loc[data['facility_type'].str.contains('tavern|pub|brew|wine_tasting|bar_grill|hooka', case=False, na=None), 'facility_type'] = 'bar'
data.loc[data['facility_type'].str.contains('bakery', case=False, na=None), 'facility_type'] = 'bakery'
data.loc[data['facility_type'].str.contains('mobil|truck|mfd', case=False, na=None), 'facility_type'] = 'mobile_food'
data.loc[data['facility_type'].str.contains('kitchen', case=False, na=None), 'facility_type'] = 'kitchen'
data.loc[data['facility_type'].str.contains('restaurant|rstaurant|diner', case=False, na=None), 'facility_type'] = 'restaurant'
data.loc[data['facility_type'].str.contains('retail', case=False, na=None), 'facility_type'] = 'retail'
data.loc[data['facility_type'].str.contains('roof', case=False, na=None), 'facility_type'] = 'rooftop'
data.loc[data['facility_type'].str.contains('grocery', case=False, na=None), 'facility_type'] = 'grocery_store'
data.loc[data['facility_type'].str.contains('liquor', case=False, na=None), 'facility_type'] = 'liquor'
data.loc[data['facility_type'].str.contains('popup', case=False, na=None), 'facility_type'] = 'popup_establishment'
data.loc[data['facility_type'].str.contains('school|college|shcool', case=False, na=None), 'facility_type'] = 'school'
data.loc[data['facility_type'].str.contains('daycare', case=False, na=None), 'facility_type'] = 'daycare'
data.loc[data['facility_type'].str.contains('cafeteria|coffee|cafe', case=False, na=None), 'facility_type'] = 'coffee'
data.loc[data['facility_type'].str.contains('drug_store|pharmacy', case=False, na=None), 'facility_type'] = 'drug_store'
data.loc[data['facility_type'].str.contains('gym|fitness|weight_loss|exercise', case=False, na=None), 'facility_type'] = 'gym'
data.loc[data['facility_type'].str.contains('commissary|machine|commiasary', case=False, na=None), 'facility_type'] = 'vending_machine'
data.loc[data['facility_type'].str.contains('ice_cream|paleteria|gelato', case=False, na=None), 'facility_type'] = 'ice_cream'
data.loc[data['facility_type'].str.contains('banquet', case=False, na=None), 'facility_type'] = 'banquet'
data.loc[data['facility_type'].str.contains('lounge', case=False, na=None), 'facility_type'] = 'lounge'
data.loc[data['facility_type'].str.contains('church|religious', case=False, na=None), 'facility_type'] = 'church'
data.loc[data['facility_type'].str.contains('kiosk', case=False, na=None), 'facility_type'] = 'kiosk'
data.loc[data['facility_type'].str.contains('health|rehab', case=False, na=None), 'facility_type'] = 'health'
data.loc[data['facility_type'].str.contains('event', case=False, na=None), 'facility_type'] = 'events'
data.loc[data['facility_type'].str.contains('donut|hotdog|hot_dog|popcorn|juice|tea|dessert|deli|salad|snack|candy|shake|watermelon|smoothie|food|sushi', case=False, na=None), 'facility_type'] = 'other_food'
data.loc[data['facility_type'].str.contains('poultry|butcher|slaughter|meat', case=False, na=None), 'facility_type'] = 'butcher'
data.loc[data['facility_type'].str.contains('profit', case=False, na=None), 'facility_type'] = 'non_profit'
data.loc[data['facility_type'].str.contains('na', case=False, na=None), 'facility_type'] = 'not_specified'
#FALTAN MUCHOS. VER R


Se reducen a 122 diferentes categorías, probablemente se puedan agrupar más

In [88]:
data.nunique()

inspection_id      215130
facility_type         122
risk                    5
zip                   113
inspection_date      2797
inspection_type        98
results                 2
violations         156570
latitude            17248
longitude           17248
dtype: int64

In [89]:
data['facility_type'].unique()

array(['not_specified', 'restaurant', 'general_store',
       'childrens_service_facility', 'grocery_store', 'gas_station',
       'bakery', 'assisted_living', 'other_food', 'school', 'catering',
       'daycare', 'ice_cream', 'liquor', 'popup_establishment', 'gym',
       'bar', 'hospital', 'kitchen', 'coffee', 'events', 'movie_theater',
       'butcher', 'wholesale', 'distribution_center', 'smokehouse',
       'other', 'banquet', 'push_carts', 'church', 'retail', 'store',
       'theater', 'shelter', 'kiosk', 'unused_storage',
       'chicago_park_district', 'pool', 'rooftop', 'night_club',
       'convenience_store', 'pantry', 'vending_machine', 'summer_feeding',
       'hotel', 'health', 'art_center', 'private_club', 'non_profit',
       'greenhouse', 'herbalife', 'dining_hall', 'drug_store',
       'repackaging_plant', 'service_gas_station', 'blockbuster_video',
       'charter', 'herbal_drinks', 'hostel', 'hair_salon', 'lounge',
       'dollar_tree', 'stadium', 'youth_housing', '

In [ ]:
# Ahora hacemos limpieza similar para columna de 'inspection_type'




dta <- dta %>% 
  # distinct(inspection_type) %>% 
  # arrange(inspection_type) %>% 
  mutate_at(vars(inspection_type), ~tolower(.)) %>% 
  mutate_at(vars(inspection_type), ~ifelse(str_detect(., 'license'),
                                         yes =  'license', no = .)) %>% 
  mutate_at(vars(inspection_type), ~ifelse(str_detect(., 'task force|taskforce'),
                                         yes =  'task force', no = .)) %>% 
  mutate_at(vars(inspection_type), ~ifelse(str_detect(., 'canvass|canvas'),
                                         yes =  'canvass', no = .)) %>% 
  mutate_at(vars(inspection_type), ~ifelse(str_detect(., 'complaint'),
                                         yes =  'complaint', no = .))

In [38]:
data['inspection_type'].unique()

array(['license', 'canvass', 'complaint', 'short_form_complaint',
       'canvass_reinspection', 'license_reinspection',
       'complaint_reinspection', 'recent_inspection',
       'suspected_food_poisoning', 'consultation', 'not_ready',
       'suspected_food_poisoning_reinspection',
       'short_form_firecomplaint', 'out_of_business', 'licensetask_force',
       'complaintfire_reinspection', 'complaintfire',
       'task_force_liquor_1475', 'tag_removal', 'package_liquor_1474',
       'finish_complaint_inspection_from_51810', 'no_entry',
       'citation_reissued', 'license_request', 'special_events_festivals',
       'possible_fbi', 'task_force_package_goods_1474', 'noninspection',
       'office_assignment', 'covid_complaint', 'fire_complaint', 'fire',
       'na', 'kitchen_closed_for_renovation', 'ob', 'corrective_action',
       'license_canceled_by_owner', 'license_consultation',
       'owner_suspended_operation_license', 'illegal_operation',
       'two_people_ate_and_got_si

In [39]:
#data[['facility_type','risk','inspection_type','violations']]=data[['facility_type','risk','inspection_type','violations']].fillna('NA')
#data['risk']=data['Zip'].fillna(0)

In [40]:
#data.isna().sum()

In [41]:
#data['violations'].isna()

In [42]:
#data['facility_type'] = data['facility_type'].lower()

In [43]:
# NO SE PUEDE POR LOS NAS ¿qué tratamiento les daremos a los na (fillna o dropna o como imputar)
#standarize_column_strings(data, ['facility_type'])
#

In [44]:
#col_text = ['dba_name',
 #'aka_name'
 'license',
 'facility_type',
 #'risk',
 #'address',
 #'city',
 #'state',
 #'zip',
 #'inspection_type',
 #'results',
 #'violations',
 #'location'
           ]


IndentationError: unexpected indent (<ipython-input-44-b3621f60200e>, line 3)

In [ ]:
#col_text

In [ ]:
#data[col_text].dtypes